In [1]:
import pandas as pd
df = pd.read_csv('../input/tencent1/11.csv')
df

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
#df = pd.read_csv('/kaggle/input/tencentforuse/edit')
df_new=pd.read_csv('/kaggle/input/addnew1/add.CSV')

In [3]:
df_new=df_new.dropna(subset=['sum']).reset_index(drop=True)
#df_new=df[:2951,:]
df_new1=df_new[['1','2','sum']]
df_new1

In [4]:
#df1 = df.dropna(subset=['play number']).reset_index(drop=True)
df1=df
df1

In [5]:
df_last = pd.concat([df1, df_new1], axis=1, join="inner")
#pd.set_option('display.max_colwidth', None)
df_last

In [6]:
df_last1 = df_last.dropna(subset=['play number']).reset_index(drop=True)
df_last1

In [7]:
def clean_number(s):
    if s[-1] == 'w':
        return float(s[:-1])
    if s[-1] == 'y':
        return float(s[:-1]) * 1e4
    return float(s)
df_last1['play number'] = df_last1['play number'].apply(clean_number)
df_last1

In [8]:
df_last1['1']=df_last1['1'].apply(pd.to_numeric,errors='coerce')
df_last1['2']=df_last1['2'].apply(pd.to_numeric,errors='coerce')
#df_last = df_last.astype({"1": 'float', "2": 'float'})
df_last1.info()
df_last1

In [9]:
df_last1['play number']
a=list(df_last1['play number'])
a.sort()
a[2*len(a)//3]

In [10]:
df_last1['play cat'] = 0
a=0
b=0
c=0
for i in range(len(df_last1)):
    n = df_last1.loc[i, 'play number']
    if n < 4015:
        df_last1.loc[i, 'play cat'] = 0
        a+=1
    elif n > 34000:
        df_last1.loc[i, 'play cat'] = 2
        c+=1
    else:
        df_last1.loc[i, 'play cat'] = 1
        b+=1
print(a,b,c)
df_last

In [11]:
df_last1.to_csv('newclasstering.csv',index=False)

In [12]:
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.model_selection import train_test_split
#test=df_last1[['status','score','year', 'topic','1','2','sum']]

In [13]:
##https://blog.csdn.net/weixin_42029733/article/details/89922575
from itertools import combinations
def combine(temp_list,n):
    temp_list2 = []
    for c in combinations(temp_list, n):
        temp_list2.append(list(c))
    return temp_list2
test=['status','year', 'topic','1','2']
end_list=[]
for i in range(len(test)):
    end_list.extend(list(combine(test,i)))
del end_list[0]
end_list.append(test)
#end_list

In [14]:
from sklearn.metrics import accuracy_score
epoch=1
accuracy=[]
for test_case in end_list: 
    cur_accuracy=0
    for i in range(epoch):
        X_train, X_test, y_train, y_test = train_test_split(
            df_last1[test_case], df_last1["play cat"], test_size=0.1
        )
        cls = LGBMClassifier()
        #cls=AutoML(mode="Perform")
        cls.fit(X_train, y_train)
        predictions = cls.predict(X_test)
        cur_accuracy+=accuracy_score(y_test, predictions)
    accuracy.append((cur_accuracy/epoch,test_case))

In [15]:
#predictions
accuracy.sort(reverse=True)
accuracy